# SciSciNet Data Processing Pipeline

This notebook processes SciSciNet data for network visualization:
- Downloads data from Hugging Face
- Filters data by institution and field
- Generates network datasets for visualization

---

## 1. Setup and Configuration


In [1]:
import os
import duckdb
from huggingface_hub import hf_hub_download
from dotenv import dotenv_values

# Configuration
config = dotenv_values()
TOKEN = config.get("HF_TOKEN")
if not TOKEN:
    raise ValueError(
        "HF_TOKEN not found in .env. Please create a .env file with HF_TOKEN=... "
    )

REPO_ID = "Northwestern-CSSI/sciscinet-v2"
DATA_DIR = "dataset"
SAMPLE_DIR = os.path.join(DATA_DIR, "sample")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(SAMPLE_DIR, exist_ok=True)

# Files to download
FILES = [
    "sciscinet_papers.parquet",
    "sciscinet_paperrefs.parquet",
    "sciscinet_paper_author_affiliation.parquet",
    "sciscinet_affiliations.parquet",
    "sciscinet_paperfields.parquet",
    "sciscinet_fields.parquet",
    "sciscinet_author_details.parquet"
]

# Processing configuration
SCHOOL_NAME_EXACT = "Beijing Normal University"
YEAR_FROM = 2015  # For full dataset (univ_cs_from2015_papers)
NETWORK_YEAR_FROM = 2020  # For network datasets (citations, coauthors, papers)
CS_FIELD_NAME = "Computer science"


## 2. Download Data from Hugging Face

Download required data files from the SciSciNet repository.


In [2]:
# Download files from Hugging Face
for filename in FILES:
    local_path = os.path.join(DATA_DIR, filename)
    if os.path.exists(local_path):
        print(f"✓ Already exists: {filename}")
        continue

    print(f"Downloading {filename}...")
    path = hf_hub_download(
        repo_id=REPO_ID,
        filename=filename,
        token=TOKEN,
        local_dir=DATA_DIR,
        resume_download=True,
        repo_type="dataset",
    )
    print(f"✓ Download completed: {os.path.basename(path)}")

print("\n✓ All data files are ready.")


✓ Already exists: sciscinet_papers.parquet
✓ Already exists: sciscinet_paperrefs.parquet
✓ Already exists: sciscinet_paper_author_affiliation.parquet
✓ Already exists: sciscinet_affiliations.parquet
✓ Already exists: sciscinet_paperfields.parquet
✓ Already exists: sciscinet_fields.parquet
✓ Already exists: sciscinet_author_details.parquet

✓ All data files are ready.


## 3. Data Overview

Inspect the structure and basic statistics of downloaded data files.


In [3]:
# Connect to DuckDB
con = duckdb.connect()

# Inspect each file
for filename in FILES:
    path = os.path.join(DATA_DIR, filename)
    print(f"\n📄 {filename}")
    
    n_rows = con.execute(f"SELECT COUNT(*) FROM read_parquet('{path}')").fetchone()[0]
    schema = con.execute(f"DESCRIBE SELECT * FROM read_parquet('{path}')").fetchdf()
    n_cols = len(schema)
    
    print(f"Shape: ({n_rows:,}, {n_cols})")
    print(f"Columns: {', '.join(schema['column_name'].tolist()[:10])}")
    if n_cols > 10:
        print(f"... and {n_cols - 10} more columns")
    
    # Show sample data
    sample = con.execute(f"SELECT * FROM read_parquet('{path}') LIMIT 3").fetchdf()
    print("\nSample data:")
    print(sample)



📄 sciscinet_papers.parquet
Shape: (249,803,279, 28)
Columns: paperid, doi, year, date, doctype, cited_by_count, is_retracted, reference_count, citation_count, C3
... and 18 more columns

Sample data:
       paperid   doi  year        date       doctype  cited_by_count  \
0  W2612738092  None  2002  2002-01-01       article               1   
1  W2952574245  None  2009  2009-04-01  book-chapter               0   
2  W3144847321  None  2006  2006-01-01  book-chapter               0   

   is_retracted  reference_count  citation_count  C3  ...  WSB_Cinf  SB_B  \
0         False                0               4   4  ...       NaN   0.0   
1         False                0               0   0  ...       NaN   NaN   
2         False                0               0   0  ...       NaN   NaN   

   SB_T  team_size  institution_count  patent_count  newsfeed_count  \
0     0          0                  0             0               0   
1  <NA>          0                  0             0        

## 4. Create Base Views

Set up DuckDB views for efficient data access.


In [4]:
# Define file paths
PAPERS = os.path.join(DATA_DIR, "sciscinet_papers.parquet")
PAA = os.path.join(DATA_DIR, "sciscinet_paper_author_affiliation.parquet")
AFF = os.path.join(DATA_DIR, "sciscinet_affiliations.parquet")
PF = os.path.join(DATA_DIR, "sciscinet_paperfields.parquet")
FIELDS = os.path.join(DATA_DIR, "sciscinet_fields.parquet")
REFS = os.path.join(DATA_DIR, "sciscinet_paperrefs.parquet")
AUTHOR_DETAILS = os.path.join(DATA_DIR, "sciscinet_author_details.parquet")

# Create views
con.execute(f"CREATE OR REPLACE VIEW papers AS SELECT * FROM read_parquet('{PAPERS}');")
con.execute(f"CREATE OR REPLACE VIEW paa AS SELECT paperid, authorid, institutionid, author_position FROM read_parquet('{PAA}');")
con.execute(f"CREATE OR REPLACE VIEW aff AS SELECT institution_id, display_name FROM read_parquet('{AFF}');")
con.execute(f"CREATE OR REPLACE VIEW pfields AS SELECT paperid, fieldid FROM read_parquet('{PF}');")
con.execute(f"CREATE OR REPLACE VIEW fields AS SELECT fieldid, display_name, level FROM read_parquet('{FIELDS}');")
con.execute(f"CREATE OR REPLACE VIEW refs AS SELECT * FROM read_parquet('{REFS}');")
con.execute(f"CREATE OR REPLACE VIEW author_details AS SELECT * FROM read_parquet('{AUTHOR_DETAILS}');")

print("✓ Base views created")


✓ Base views created


## 5. Identify Target Institution

Match the target institution (Beijing Normal University) from the affiliations table.


In [5]:
# Find institution ID
inst_df = con.execute("""
    SELECT DISTINCT institution_id
    FROM aff
    WHERE display_name = ?
""", [SCHOOL_NAME_EXACT]).fetchdf()

if len(inst_df) != 1:
    raise ValueError(f"Institution match error: {SCHOOL_NAME_EXACT} -> {len(inst_df)} matches found")

MY_INST_ID = inst_df.iloc[0]["institution_id"]
print(f"✓ Institution matched: {SCHOOL_NAME_EXACT} -> {MY_INST_ID}")

# Create table for filtering
con.execute("CREATE OR REPLACE TABLE my_inst AS SELECT ?::VARCHAR AS institution_id", [MY_INST_ID])


✓ Institution matched: Beijing Normal University -> I25254941


## 6. Identify Computer Science Field

Match the Computer Science field ID from the fields table.


In [6]:
# Find Computer Science field ID
cs_df = con.execute("""
    SELECT DISTINCT fieldid
    FROM fields
    WHERE display_name = ?
""", [CS_FIELD_NAME]).fetchdf()

if len(cs_df) != 1:
    raise ValueError(f"Field match error: '{CS_FIELD_NAME}' -> {len(cs_df)} matches found")

CS_FIELD_ID = cs_df.iloc[0]["fieldid"]
print(f"✓ Field matched: {CS_FIELD_NAME} -> {CS_FIELD_ID}")

# Create table for filtering
con.execute("CREATE OR REPLACE TABLE cs_field_ids AS SELECT ?::VARCHAR AS fieldid", [CS_FIELD_ID])

# Create CS papers set
con.execute("""
    CREATE OR REPLACE TABLE cs_papers AS
    SELECT DISTINCT pfields.paperid
    FROM pfields
    JOIN cs_field_ids USING(fieldid);
""")

✓ Field matched: Computer science -> C41008148


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

## 7. Generate Citation Network Data (2020+)

Create internal citation network data (papers citing other papers within the same dataset) from 2020 onwards.


In [7]:
# Generate internal citations (both citing and cited papers are in our dataset and from 2020+)
con.execute(f"""
    CREATE OR REPLACE TABLE citations_5y_internal AS
    WITH univ_cs_papers_2020 AS (
        SELECT DISTINCT p.paperid, p.year
        FROM papers p
        JOIN paa ON p.paperid = paa.paperid
        JOIN my_inst mi ON paa.institutionid = mi.institution_id
        JOIN cs_papers csp ON p.paperid = csp.paperid
        WHERE p.year >= {NETWORK_YEAR_FROM}
    )
    SELECT 
        r.citing_paperid AS citing_paperid,
        r.cited_paperid AS cited_paperid,
        p.year AS year
    FROM refs r
    JOIN univ_cs_papers_2020 p ON r.citing_paperid = p.paperid
    WHERE r.cited_paperid IN (SELECT paperid FROM univ_cs_papers_2020);
""")

# Export
OUT_CITATIONS = os.path.join(SAMPLE_DIR, "citations_5y_internal.parquet")
con.execute(f"COPY citations_5y_internal TO '{OUT_CITATIONS}' (FORMAT PARQUET);")

n_citations = con.execute("SELECT COUNT(*) FROM citations_5y_internal").fetchone()[0]
print(f"✓ citations_5y_internal: {n_citations:,} rows")
print(f"💾 Saved: {OUT_CITATIONS}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✓ citations_5y_internal: 6,868 rows
💾 Saved: dataset/sample/citations_5y_internal.parquet


## 8. Generate Coauthor Network Data (2020+)

Create coauthor relationship data from 2020 onwards.


In [8]:
# Generate coauthor details (paperid, authorid, author_position, year) - only 2020+
con.execute(f"""
    CREATE OR REPLACE TABLE coauthor_details_5y AS
    WITH univ_cs_papers_2020 AS (
        SELECT DISTINCT p.paperid, p.year
        FROM papers p
        JOIN paa ON p.paperid = paa.paperid
        JOIN my_inst mi ON paa.institutionid = mi.institution_id
        JOIN cs_papers csp ON p.paperid = csp.paperid
        WHERE p.year >= {NETWORK_YEAR_FROM}
    )
    SELECT 
        paa.paperid,
        paa.authorid,
        paa.author_position,
        p.year
    FROM paa
    JOIN univ_cs_papers_2020 p ON paa.paperid = p.paperid
    JOIN my_inst mi ON paa.institutionid = mi.institution_id;
""")

# Export
OUT_COAUTHORS = os.path.join(SAMPLE_DIR, "coauthor_details_5y.parquet")
con.execute(f"COPY coauthor_details_5y TO '{OUT_COAUTHORS}' (FORMAT PARQUET);")

n_coauthors = con.execute("SELECT COUNT(*) FROM coauthor_details_5y").fetchone()[0]
print(f"✓ coauthor_details_5y: {n_coauthors:,} rows")
print(f"💾 Saved: {OUT_COAUTHORS}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✓ coauthor_details_5y: 30,806 rows
💾 Saved: dataset/sample/coauthor_details_5y.parquet


## 9. Generate Author Dataset (2020+)

Create author information dataset for authors in the filtered papers from 2020 onwards.


In [9]:
# Get unique authors from coauthor details
con.execute("""
    CREATE OR REPLACE TABLE authors_university_5y_cs AS
    SELECT DISTINCT
        ad.authorid,
        ad.display_name AS author_name,
        ad.orcid,
        ad.works_count,
        ad.cited_by_count,
        paa.institutionid,
        aff.display_name AS institution_name
    FROM coauthor_details_5y cad
    JOIN author_details ad ON cad.authorid = ad.authorid
    JOIN paa ON cad.paperid = paa.paperid AND cad.authorid = paa.authorid
    JOIN aff ON paa.institutionid = aff.institution_id
    WHERE paa.institutionid = ?::VARCHAR;
""", [MY_INST_ID])

# Export
OUT_AUTHORS = os.path.join(SAMPLE_DIR, "authors_university_5y_cs.parquet")
con.execute(f"COPY authors_university_5y_cs TO '{OUT_AUTHORS}' (FORMAT PARQUET);")

n_authors = con.execute("SELECT COUNT(*) FROM authors_university_5y_cs").fetchone()[0]
print(f"✓ authors_university_5y_cs: {n_authors:,} rows")
print(f"💾 Saved: {OUT_AUTHORS}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✓ authors_university_5y_cs: 12,700 rows
💾 Saved: dataset/sample/authors_university_5y_cs.parquet


## 10. Generate Papers Dataset (2020+)

Create papers dataset for the filtered set from 2020 onwards.


In [10]:
# Papers from 2020 onwards
con.execute(f"""
    CREATE OR REPLACE TABLE papers_university_5y_cs AS
    SELECT DISTINCT p.*
    FROM papers p
    JOIN paa ON p.paperid = paa.paperid
    JOIN my_inst mi ON paa.institutionid = mi.institution_id
    JOIN cs_papers csp ON p.paperid = csp.paperid
    WHERE p.year >= {NETWORK_YEAR_FROM};
""")

# Export
OUT_PAPERS = os.path.join(SAMPLE_DIR, "papers_university_5y_cs.parquet")
con.execute(f"COPY papers_university_5y_cs TO '{OUT_PAPERS}' (FORMAT PARQUET);")

n_papers = con.execute("SELECT COUNT(*) FROM papers_university_5y_cs").fetchone()[0]
print(f"✓ papers_university_5y_cs: {n_papers:,} rows")
print(f"💾 Saved: {OUT_PAPERS}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✓ papers_university_5y_cs: 13,310 rows
💾 Saved: dataset/sample/papers_university_5y_cs.parquet


## 11. Generate University CS Papers Dataset (2015+)

Create a filtered dataset containing all papers from the target institution in Computer Science from 2015 onwards.


In [11]:
# Create and export univ_cs_from2015_papers table
con.execute(f"""
    CREATE OR REPLACE TABLE univ_cs_from2015_papers AS
    SELECT DISTINCT p.*
    FROM papers p
    JOIN paa ON p.paperid = paa.paperid
    JOIN my_inst mi ON paa.institutionid = mi.institution_id
    JOIN cs_papers csp ON p.paperid = csp.paperid
    WHERE p.year >= {YEAR_FROM};
""")

# Export
OUT = os.path.join(SAMPLE_DIR, "univ_cs_from2015_papers.parquet")
con.execute(f"COPY univ_cs_from2015_papers TO '{OUT}' (FORMAT PARQUET);")

# Statistics
n = con.execute("SELECT COUNT(*) FROM univ_cs_from2015_papers").fetchone()[0]
print(f"✓ Created and exported univ_cs_from2015_papers: {n:,} rows")
print(f"💾 Saved: {OUT}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✓ Created and exported univ_cs_from2015_papers: 20,770 rows
💾 Saved: dataset/sample/univ_cs_from2015_papers.parquet
